In this notebook we perform collaborative filtering using gradient descent (without for loops) on the movielens dataset.

In [5]:
import numpy as np
import pandas as pd
from scipy import sparse

## Helper Functions:

In [3]:
def proc_col(col):
    """
    Function to encode a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o: i for i, o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)


def encode_data(df):
    """
    Function to encode rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      df: Dataframe with columns user_id,movie_id,rating 
    
    Returns:
      df: dataframe with the encode data
      num_users
      num_movies
    """
    user_encoded, df['userId'], num_users = proc_col(df.userId)
    movie_encoded, df['movieId'], num_movies = proc_col(df.movieId)
    return df, num_users, num_movies

## Collaborative Filtering with Gradient Descent:

Collaborative filtering in matrix factorization form can be represented as $Y = UV^T$ where 
- Y is the matrix of ratings with rows corresponding to users and columns to movies. This is also known as utility matrix
- U is the user embedding matrix with each row capturing one user's features
- V is the movie embedding matrix with each row capturing one movies's features

An alternate representation would have biases corresponding to movies, users and can be represented as $Y = UV^T + U_0 + V_0^T$ where
- $U_0$ is a column vector with user biases
- $V_0$ is a column vector with movie biases

Each element in the utility matrix (rating of $j^{th}$ movie by $i^{th}$ user) can be computed using the below formula
$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$

We will train the embeddings using gradient descent and MSE as loss function. Corresponding gradient descent equations can be written as:

$$u_{ik} = u_{ik} + \frac{2\eta}{N} \sum_{j:r_{ij}=1} (y_{ij} - u_{i} . v_{j} - u_{0i} - v_{0j})v_{jk} $$
$$v_{jk} = v_{jk} + \frac{2\eta}{N} \sum_{j:r_{ij}=1} (y_{ij} - u_{i} . v_{j} - u_{0i} - v_{0j})u_{ik} $$
$$u_{0i} = u_{0i} + \frac{2\eta}{N} \sum_{j:r_{ij}=1} (y_{ij} - u_{i} . v_{j} - u_{0i} - v_{0j})$$
$$v_{0j} = v_{0j} + \frac{2\eta}{N} \sum_{j:r_{ij}=1} (y_{ij} - u_{i} . v_{j} - u_{0i} - v_{0j})$$

The same equations acan be written in vectorized form as 

$$\Delta = (Y - U . V^{T} - U_{0} - V_{0}^{T}) \otimes R $$

Where 
- $U_{0}$ and $V_{0}$ are bias matrices of order $n_{u} \times 1$ and $n_{m} \times 1$ respectively. 
- R is an $n_{u} \times n_{m}$ matrix with element $r_{ij}$

Numpy broadcasting takes care of the dimension mismatch between $Y$, $U_{0}$ and $V_{0}$

$$ U = U + \frac{2\eta}{N}\Delta . V$$
$$ V = V + \frac{2\eta}{N}\Delta^{T} . U$$
$$ U_{0} = U_{0} + \frac{2\eta}{N}\Delta . I_{v}$$
$$ V_{0} = V_{0} + \frac{2\eta}{N}\Delta^{T} . I_{u}$$

where $I_{u}$ and $I_{v}$ are unit matrices of shape same as $U_{0}$ and $V_{0}$

### Initialize Embeddings

In [4]:
def create_embedings(n, K):
    """
    Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6 * np.random.random((n, K)) / K
    return emb

### Encoding Ratings as a Sparse Matrix

In [9]:
def df2matrix(df, nrows, ncols, column_name="rating"):
    """
    Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix(
        (values, (ind_user, ind_movie)), shape=(nrows, ncols))


def sparse_multiply(df, emb_user, emb_movie):
    """
    This function returns U*V^T element wise multi by R as a sparse matrix.
    
    It avoids creating the dense matrix U*V^T
    """
    df["Prediction"] = np.sum(
        emb_user[df["userId"].values] * emb_movie[df["movieId"].values],
        axis=1)
    return df2matrix(
        df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

In [13]:
def encode_new_data(df_val, df_train):
    """
    Encodes df_val with the same encoding as df_train.
    
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    user_encoded, df_train['userId'], num_users = proc_col(df_train.userId)
    movie_encoded, df_train['movieId'], num_movies = proc_col(df_train.movieId)  
    
    df_val['userId'] = df_val.userId.apply(lambda x: user_encoded.get(x, -1)).astype('int64')
    df_val['movieId'] = df_val.movieId.apply(lambda x: movie_encoded.get(x, -1)).astype('int64')
    df_val = df_val.drop(df_val[(df_val['userId'] == -1) | (df_val['movieId'] == -1)].index)
    
    return df_val

### Define Cost Function

In [10]:
def cost(df, emb_user, emb_movie):
    """
    Computes mean square error
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    ratings = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    predictions = sparse_multiply(df, emb_user, emb_movie)
    error = (ratings - predictions).power(2).sum() / ratings.count_nonzero()
    return error

### Define Gradient Descent Functions

In [16]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """
    Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1) / e


def gradient(df, Y, emb_user, emb_movie):
    """
    Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    N = Y.count_nonzero()
    predictions = sparse_multiply(df, emb_user, emb_movie)
    delta = Y - predictions
    d_emb_user = -(2.0 / N) * delta * emb_movie
    d_emb_movie = -(2.0 / N) * delta.T * emb_user
    return d_emb_user, d_emb_movie


def gradient_descent(df,
                     emb_user,
                     emb_movie,
                     iterations=100,
                     learning_rate=0.01,
                     df_val=None):
    """
    Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 100 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    v_user, v_movie = gradient(df, Y, emb_user, emb_movie)
    for i in range(1, iterations + 1):
        grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)
        v_user, v_movie = (0.9 * v_user + 0.1 * grad_user), (
            0.9 * v_movie + 0.1 * grad_movie)
        emb_user -= learning_rate * v_user
        emb_movie -= learning_rate * v_movie

        if i % 100 == 0:
            train_cost = cost(df, emb_user, emb_movie)
            print("Training Cost: %.4f" % train_cost)
            if df_val is not None:
                val_cost = cost(df_val, emb_user, emb_movie)
                print("Validation Cost: %.4f" % val_cost)

    return emb_user, emb_movie

### Define Regularized Gradient Descent Functions

In [19]:
def cost_reg(df, emb_user, emb_movie, reg_param):
    """
    Computes mean square error
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      reg_param: regularization parameter
      
    Returns:
      error(float): this is the MSE
    """
    ratings = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    predictions = sparse_multiply(df, emb_user, emb_movie)
    mse = (ratings-predictions).power(2).sum() / ratings.count_nonzero() +\
    reg_param * (np.sum(np.square(emb_user)) + np.sum(np.square(emb_movie)))
    return mse

def gradient_reg(df, Y, emb_user, emb_movie, reg_param):
    """
    Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      reg_param: regularization parameter
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    N = Y.count_nonzero()
    predictions = sparse_multiply(df, emb_user, emb_movie)
    delta = Y - predictions
    d_emb_user = -(2.0/N)*delta*emb_movie + 2.0*reg_param*emb_user
    d_emb_movie = -(2.0/N)*delta.T*emb_user + 2.0*reg_param*emb_movie
    return d_emb_user, d_emb_movie

def gradient_descent_reg(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None, reg_param=0):
    """
    Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 100 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    num_estimates = (emb_user.shape[0] + emb_movie.shape[0]) * emb_user.shape[1]
    rp = reg_param/num_estimates
    v_user, v_movie = gradient_reg(df, Y, emb_user, emb_movie, rp)
    for i in range(0,iterations):
        grad_user, grad_movie = gradient_reg(df, Y, emb_user, emb_movie, rp)
        v_user, v_movie = (0.9*v_user + 0.1*grad_user), (0.9*v_movie + 0.1*grad_movie)
        emb_user -= learning_rate*v_user
        emb_movie -= learning_rate*v_movie
    
        if i%100 == 0:
            train_cost = cost_reg(df, emb_user, emb_movie, rp)
            print("Training Cost: %.4f" %train_cost)
            if df_val is not None:
                val_cost = cost_reg(df_val, emb_user, emb_movie, rp)
                print("Validation Cost: %.4f" %val_cost)
    
    return emb_user, emb_movie

### Load Data and Train-Test Split:

In [14]:
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")

In [15]:
# Random split as time based split may lead to cold start problem
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8 
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())

In [17]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

Training Cost: 7.2257
Validation Cost: 7.3617
Training Cost: 4.0767
Validation Cost: 4.2013
Training Cost: 2.8326
Validation Cost: 2.9361
Training Cost: 2.1876
Validation Cost: 2.2800
Training Cost: 1.8059
Validation Cost: 1.8946
Training Cost: 1.5582
Validation Cost: 1.6467
Training Cost: 1.3859
Validation Cost: 1.4761
Training Cost: 1.2600
Validation Cost: 1.3528
Training Cost: 1.1644
Validation Cost: 1.2603
Training Cost: 1.0895
Validation Cost: 1.1889
Training Cost: 1.0293
Validation Cost: 1.1324
Training Cost: 0.9800
Validation Cost: 1.0870
Training Cost: 0.9388
Validation Cost: 1.0498
Training Cost: 0.9038
Validation Cost: 1.0190
Training Cost: 0.8736
Validation Cost: 0.9931
Training Cost: 0.8473
Validation Cost: 0.9712
Training Cost: 0.8240
Validation Cost: 0.9525
Training Cost: 0.8030
Validation Cost: 0.9365
Training Cost: 0.7840
Validation Cost: 0.9226
Training Cost: 0.7666
Validation Cost: 0.9104


In [20]:
K = 50
emb_user_reg = create_embedings(num_users, K)
emb_movie_reg = create_embedings(num_movies, K)
emb_user_reg, emb_movie_reg = gradient_descent_reg(df_train, emb_user_reg, emb_movie_reg, iterations=2000, learning_rate=1, df_val=df_val, reg_param=10)

Training Cost: 12.4318
Validation Cost: 12.5388
Training Cost: 7.2795
Validation Cost: 7.4154
Training Cost: 4.2089
Validation Cost: 4.3336
Training Cost: 3.0034
Validation Cost: 3.1070
Training Cost: 2.3839
Validation Cost: 2.4764
Training Cost: 2.0212
Validation Cost: 2.1098
Training Cost: 1.7884
Validation Cost: 1.8766
Training Cost: 1.6286
Validation Cost: 1.7181
Training Cost: 1.5131
Validation Cost: 1.6050
Training Cost: 1.4265
Validation Cost: 1.5212
Training Cost: 1.3594
Validation Cost: 1.4574
Training Cost: 1.3063
Validation Cost: 1.4077
Training Cost: 1.2632
Validation Cost: 1.3682
Training Cost: 1.2277
Validation Cost: 1.3364
Training Cost: 1.1978
Validation Cost: 1.3104
Training Cost: 1.1724
Validation Cost: 1.2890
Training Cost: 1.1503
Validation Cost: 1.2711
Training Cost: 1.1310
Validation Cost: 1.2560
Training Cost: 1.1138
Validation Cost: 1.2433
Training Cost: 1.0982
Validation Cost: 1.2324


### Model Performance

In [18]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

0.7665730026459857 0.9104271876851561


In [25]:
reg_param = 10
num_estimates = (emb_user_reg.shape[0] + emb_movie_reg.shape[0]) * emb_user_reg.shape[1]
rp = reg_param/num_estimates
train_mse_reg = cost_reg(df_train, emb_user_reg, emb_movie_reg, rp)
val_mse_reg = cost_reg(df_val, emb_user_reg, emb_movie_reg, rp)
print(train_mse_reg, val_mse_reg)

1.0841202174680702 1.2232326960254476
